In [1]:
import boto3
import time

In [2]:
import pandas as pd
from io import StringIO

In [17]:
factCovid = pd.read_csv('factCovid.csv')

In [21]:
factCovid.head()

,fips,province_state,country_region,confirmed,deaths,recovered,active,date,state,positive,negative,hospitalizedcurrently,hospitalized,hospitalizeddischarged
0,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210219,PR,98624.0,305972.0,214.0,NaN,NaN
1,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210218,PR,98506.0,305972.0,233.0,NaN,NaN
2,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210217,PR,98428.0,305972.0,241.0,NaN,NaN
3,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210216,PR,98317.0,305972.0,220.0,NaN,NaN
4,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210215,PR,97837.0,305972.0,209.0,NaN,NaN


In [20]:
factCovid.drop('Unnamed: 0',axis=1,inplace=True)

In [22]:
factCovidSchema = pd.io.sql.get_schema(factCovid.reset_index(),'factCovid')

In [23]:
print("".join(factCovidSchema))

CREATE TABLE "factCovid" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "state" TEXT,
  "positive" REAL,
  "negative" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)


In [25]:
#Initiating Config

In [47]:
import configparser

In [60]:
config = configparser.ConfigParser()

In [61]:
config.read('cluster.config')

['cluster.config']

In [62]:
config['POSTGRES']['PG_HOST']

'172.17.0.2'

In [63]:
db = config['POSTGRES']['PG_DB']
user = config['POSTGRES']['PG_UNAME']
passwd = config['POSTGRES']['PG_PASS']
port = config['POSTGRES']['PG_PORT']
host = config['POSTGRES']['PG_HOST']

In [69]:
import psycopg2

In [71]:
try:
    conn = psycopg2.connect(host=host,dbname=db,user=user,password=passwd,port=port)
except Exception as e:
    print(e)

In [72]:
conn.set_session(autocommit=True)

In [73]:
try:
    cur = conn.cursor()
    
except:
    print(e)

In [81]:
try:
    cur.execute("""
        CREATE TABLE "factcovid" (
        "index" NUMERIC,
          "fips" NUMERIC,
          "province_state" VARCHAR(255),
          "country_region" VARCHAR(255),
          "confirmed" NUMERIC,
          "deaths" NUMERIC,
          "recovered" NUMERIC,
          "active" NUMERIC,
          "date" NUMERIC,
          "state" VARCHAR(255),
          "positive" NUMERIC,
          "negative" NUMERIC,
          "hospitalizedcurrently" NUMERIC,
          "hospitalized" NUMERIC,
          "hospitalizeddischarged" NUMERIC
        )
    """)
except Exception as e:
    print(e)

In [83]:
try:
    cur.execute("""
        COPY factcovid from '/var/lib/postgresql/data/factCovid.csv'
        DELIMITER ',' CSV HEADER
    """)
except Exception as e:
    print(e)

In [84]:
from pyspark.sql import SparkSession

In [85]:
spark = SparkSession.builder.appName('dimreg1').getOrCreate()

22/10/31 19:48:06 WARN Utils: Your hostname, codeStation resolves to a loopback address: 127.0.1.1; using 192.168.36.84 instead (on interface wlo1)
22/10/31 19:48:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/31 19:48:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [94]:
dimreg = spark.read.option('header','true').csv('dimReg1.csv',inferSchema=True)

In [91]:
dimreg.head(2)

22/10/31 19:50:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , fips, province_state, country_region, latitude, longitude
 Schema: _c0, fips, province_state, country_region, latitude, longitude
Expected: _c0 but found: 
CSV file: file:///run/media/solverbot/repoA/gitFolders/DEtheHardWay/dimReg1.csv


[Row(_c0='0', fips=None, province_state='Anhui', country_region='China', latitude='31.826', longitude='117.226'),
 Row(_c0='1', fips=None, province_state='Beijing', country_region='China', latitude='40.182', longitude='116.414')]

In [95]:
dimreg.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- fips: double (nullable = true)
 |-- province_state: string (nullable = true)
 |-- country_region: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [96]:
dimreg.columns

['_c0', 'fips', 'province_state', 'country_region', 'latitude', 'longitude']

In [106]:
dimreg.select(['fips']).show()

+----+
|fips|
+----+
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
|null|
+----+
only showing top 20 rows



In [99]:
dimreg.select(['latitude','longitude']).show()

+--------+---------+
|latitude|longitude|
+--------+---------+
|  31.826|  117.226|
|  40.182|  116.414|
|  30.057|  107.874|
|  26.079|  117.987|
|  36.061|  103.834|
|  23.342|  113.424|
|   23.83|  108.788|
|  26.815|  106.875|
|  19.196|  109.745|
|  38.043|  114.515|
|  47.862|  127.761|
|  33.882|  113.614|
|    22.3|    114.2|
|  30.976|  112.271|
|   27.61|  111.709|
|  44.093|  113.945|
|  32.971|  119.455|
|  27.614|  115.722|
|  43.666|  126.192|
|  41.296|  122.609|
+--------+---------+
only showing top 20 rows



In [100]:
dimreg.describe().show()

22/10/31 19:54:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , fips, province_state, country_region, latitude, longitude
 Schema: _c0, fips, province_state, country_region, latitude, longitude
Expected: _c0 but found: 
CSV file: file:///run/media/solverbot/repoA/gitFolders/DEtheHardWay/dimReg1.csv


+-------+------------------+------------------+--------------+--------------------+------------------+-----------------+
|summary|               _c0|              fips|province_state|      country_region|          latitude|        longitude|
+-------+------------------+------------------+--------------+--------------------+------------------+-----------------+
|  count|            222804|            198364|        208218|              222804|            222361|           222419|
|   mean|          111401.5|31285.762537557217|          null|                null| 36.22746800473025|-78.0409227853748|
| stddev|64318.119025668035|17118.692319266127|          null|                null|11.122844262895208|45.34682695389383|
|    min|                 0|              72.0|   """Bonaire"| Sint Eustatius a...|           -52.368|         -170.132|
|    max|            222803|           99999.0|      Zhejiang|            Zimbabwe|            71.707|          178.065|
+-------+------------------+----

In [102]:
dimreg1 = dimreg.withColumn('fips_P2', dimreg['fips']+2)

In [103]:
dimreg1.head(2)

22/10/31 19:56:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , fips, province_state, country_region, latitude, longitude
 Schema: _c0, fips, province_state, country_region, latitude, longitude
Expected: _c0 but found: 
CSV file: file:///run/media/solverbot/repoA/gitFolders/DEtheHardWay/dimReg1.csv


[Row(_c0=0, fips=None, province_state='Anhui', country_region='China', latitude=31.826, longitude=117.226, fips_P2=None),
 Row(_c0=1, fips=None, province_state='Beijing', country_region='China', latitude=40.182, longitude=116.414, fips_P2=None)]

In [112]:
dimreg2 = dimreg1.drop('fips','fips_P2')

In [113]:
dimreg2.head(2)

22/10/31 20:02:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , province_state, country_region, latitude, longitude
 Schema: _c0, province_state, country_region, latitude, longitude
Expected: _c0 but found: 
CSV file: file:///run/media/solverbot/repoA/gitFolders/DEtheHardWay/dimReg1.csv


[Row(_c0=0, province_state='Anhui', country_region='China', latitude=31.826, longitude=117.226),
 Row(_c0=1, province_state='Beijing', country_region='China', latitude=40.182, longitude=116.414)]

In [116]:
dimreg3 = dimreg2.withColumnRenamed('_c0','fips')

In [117]:
dimreg3.head(2)

22/10/31 20:05:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , province_state, country_region, latitude, longitude
 Schema: _c0, province_state, country_region, latitude, longitude
Expected: _c0 but found: 
CSV file: file:///run/media/solverbot/repoA/gitFolders/DEtheHardWay/dimReg1.csv


[Row(fips=0, province_state='Anhui', country_region='China', latitude=31.826, longitude=117.226),
 Row(fips=1, province_state='Beijing', country_region='China', latitude=40.182, longitude=116.414)]

In [118]:
dimreg3.na.drop().show()

22/10/31 20:06:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , province_state, country_region, latitude, longitude
 Schema: _c0, province_state, country_region, latitude, longitude
Expected: _c0 but found: 
CSV file: file:///run/media/solverbot/repoA/gitFolders/DEtheHardWay/dimReg1.csv
+----+--------------+--------------+--------+---------+
|fips|province_state|country_region|latitude|longitude|
+----+--------------+--------------+--------+---------+
|   0|         Anhui|         China|  31.826|  117.226|
|   1|       Beijing|         China|  40.182|  116.414|
|   2|     Chongqing|         China|  30.057|  107.874|
|   3|        Fujian|         China|  26.079|  117.987|
|   4|         Gansu|         China|  36.061|  103.834|
|   5|     Guangdong|         China|  23.342|  113.424|
|   6|       Guangxi|         China|   23.83|  108.788|
|   7|       Guizhou|         China|  26.815|  106.875|
|   8|           Hai|         China|  19.196|  109.745|
|   9|   

In [120]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCol='latitude',
    outputCol="lat_imputed",
    ).setStrategy("mean")

In [121]:
imputer.fit(dimreg3).transform(dimreg3).show()

22/10/31 20:13:14 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , province_state, country_region, latitude, longitude
 Schema: _c0, province_state, country_region, latitude, longitude
Expected: _c0 but found: 
CSV file: file:///run/media/solverbot/repoA/gitFolders/DEtheHardWay/dimReg1.csv
+----+--------------+--------------+--------+---------+-----------+
|fips|province_state|country_region|latitude|longitude|lat_imputed|
+----+--------------+--------------+--------+---------+-----------+
|   0|         Anhui|         China|  31.826|  117.226|     31.826|
|   1|       Beijing|         China|  40.182|  116.414|     40.182|
|   2|     Chongqing|         China|  30.057|  107.874|     30.057|
|   3|        Fujian|         China|  26.079|  117.987|     26.079|
|   4|         Gansu|         China|  36.061|  103.834|     36.061|
|   5|     Guangdong|         China|  23.342|  113.424|     23.342|
|   6|       Guangxi|         China|   23.83|  108.788|      23.83|
|

In [122]:
#filter operations

In [124]:
dimhospital = spark.read.option('header','true').csv('dimhospital.csv',inferSchema=True)

In [126]:
dimhospital.head(2)

22/10/31 20:18:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , fips, state_name, hospital_name, hospital_type, hq_address, hq_city, hq_state, latitude, longtitude
 Schema: _c0, fips, state_name, hospital_name, hospital_type, hq_address, hq_city, hq_state, latitude, longtitude
Expected: _c0 but found: 
CSV file: file:///run/media/solverbot/repoA/gitFolders/DEtheHardWay/dimhospital.csv


[Row(_c0=0, fips=4013.0, state_name='Arizona', hospital_name='Phoenix VA Health Care System (AKA Carl T Hayden VA Medical Center)', hospital_type='VA Hospital', hq_address='650 E Indian School Rd', hq_city='Phoenix', hq_state='AZ', latitude=33.49549780000007, longtitude=-112.06615689999995),
 Row(_c0=1, fips=4019.0, state_name='Arizona', hospital_name='Southern Arizona VA Health Care System', hospital_type='VA Hospital', hq_address='3601 S 6th Ave', hq_city='Tucson', hq_state='AZ', latitude=32.181263400000034, longtitude=-110.96588519999996)]

In [129]:
dimhospital.filter("fips >= 2000").show()

22/10/31 20:19:56 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , fips, state_name, hospital_name, hospital_type, hq_address, hq_city, hq_state, latitude, longtitude
 Schema: _c0, fips, state_name, hospital_name, hospital_type, hq_address, hq_city, hq_state, latitude, longtitude
Expected: _c0 but found: 
CSV file: file:///run/media/solverbot/repoA/gitFolders/DEtheHardWay/dimhospital.csv
+---+-------+--------------------+--------------------+--------------------+--------------------+-----------+--------+------------------+-------------------+
|_c0|   fips|          state_name|       hospital_name|       hospital_type|          hq_address|    hq_city|hq_state|          latitude|         longtitude|
+---+-------+--------------------+--------------------+--------------------+--------------------+-----------+--------+------------------+-------------------+
|  0| 4013.0|             Arizona|Phoenix VA Health...|         VA Hospital|650 E Indian Scho...|    Phoeni

In [132]:
dimhospital.filter((dimhospital['fips']<=5700) & (dimhospital['fips'] > 2000)).show()

22/10/31 20:22:22 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , fips, state_name, hospital_name, hospital_type, hq_address, hq_city, hq_state, latitude, longtitude
 Schema: _c0, fips, state_name, hospital_name, hospital_type, hq_address, hq_city, hq_state, latitude, longtitude
Expected: _c0 but found: 
CSV file: file:///run/media/solverbot/repoA/gitFolders/DEtheHardWay/dimhospital.csv
+---+------+----------+--------------------+--------------------+--------------------+-------------+--------+------------------+-------------------+
|_c0|  fips|state_name|       hospital_name|       hospital_type|          hq_address|      hq_city|hq_state|          latitude|         longtitude|
+---+------+----------+--------------------+--------------------+--------------------+-------------+--------+------------------+-------------------+
|  0|4013.0|   Arizona|Phoenix VA Health...|         VA Hospital|650 E Indian Scho...|      Phoenix|      AZ| 33.49549780000007|-112.0

In [134]:
regulation = spark.read.csv('dimReg2.csv',header=True, inferSchema=True)

In [135]:
regulation.head(2)

22/10/31 20:27:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , fips, county, state
 Schema: _c0, fips, county, state
Expected: _c0 but found: 
CSV file: file:///run/media/solverbot/repoA/gitFolders/DEtheHardWay/dimReg2.csv


[Row(_c0=0, fips=53077.0, county='Yakima', state='Washington'),
 Row(_c0=1, fips=54001.0, county='Barbour', state='West Virginia')]

In [137]:
regulation.groupBy('county').count().show()

+--------------+-----+
|        county|count|
+--------------+-----+
|         Tyler| 1537|
|East Feliciana|  779|
|     Worcester| 1582|
|        Aitkin|  757|
|     Petroleum|  586|
|      Thurston| 1543|
|         Bucks|  794|
|        Grimes|  780|
|          Utah|  790|
|       Hanover|  793|
|         Izard|  768|
|         Pasco|  795|
|        Hawaii|  788|
|      Montcalm|  793|
|     Oktibbeha|  783|
|       Shannon|  717|
|    Deer Lodge|  776|
|       Gilliam|  679|
|        Caguas|  739|
|    Charleston|  799|
+--------------+-----+
only showing top 20 rows



In [138]:
regulation.groupBy('state').sum().show()

22/10/31 20:29:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , fips, state
 Schema: _c0, fips, state
Expected: _c0 but found: 
CSV file: file:///run/media/solverbot/repoA/gitFolders/DEtheHardWay/dimReg2.csv


+--------------------+------------+-------------+
|               state|    sum(_c0)|    sum(fips)|
+--------------------+------------+-------------+
|                Utah| 28780298919| 1.08570141E9|
|              Hawaii|  4760226343|  5.5338432E7|
|           Minnesota| 84873200010|1.817647672E9|
|                Ohio| 85860935839|2.683853682E9|
|Northern Mariana ...|  2391754508|  1.0035327E8|
|            Arkansas| 73502504741| 2.95607797E8|
|              Oregon| 34583426977|1.139855057E9|
|               Texas|244263926956|9.336975802E9|
|        North Dakota| 51560439334| 1.51383607E9|
|        Pennsylvania| 65003760414|2.205951373E9|
|         Connecticut|  8730197645|  5.6894406E7|
|            Nebraska| 88301904981|2.137219674E9|
|             Vermont| 14582546321| 5.48953718E8|
|      American Samoa|   251491011|         null|
|              Nevada| 16156226090| 4.08602573E8|
|         Puerto Rico| 75652585523|4.151978944E9|
|          Washington| 38210825916|1.628990281E9|


In [ ]:
regulation.agg({ })

In [139]:
electricityGeneration = spark.read.csv('Electricity generation.csv',header=True,inferSchema=True)

In [140]:
electricityGeneration.show()

+-------+----+-------+-----+-----------+--------+-----+-------+-------+----------+--------+-------------+----+------+-------------+
|Country|Year|   Coal|  Oil|Natural gas|Biofuels|Waste|Nuclear|  Hydro|Geothermal|Solar PV|Solar thermal|Tide|  Wind|Other sources|
+-------+----+-------+-----+-----------+--------+-----+-------+-------+----------+--------+-------------+----+------+-------------+
|    USA|2018|1254542|38825|    1513414|   58955|18259| 841329| 315619|     18962|   87183|         4640|null|277918|         5263|
|     UK|2018|  17570| 1682|     131482|   32086| 8023|  65064|   7963|     57116|   12922|         null|   8|  null|         null|
|    SWE|2018|   2035|  379|        564|    9338| 2539|  65801|  61605|     16623|     404|         null|null|  null|         null|
|    RUS|2017| 174568| 6976|     518660|      84| 2594| 203143| 187131|       435|     558|         null|null|   140|         null|
|    CHN|2017|4508568|10278|     196400|   79537|13386| 248070|1189840|     

In [141]:
electricity_earth = electricityGeneration.select(['Year','Coal','Oil'])

In [143]:
electricity_earth.head(5)

[Row(Year=2018, Coal=1254542, Oil=38825),
 Row(Year=2018, Coal=17570, Oil=1682),
 Row(Year=2018, Coal=2035, Oil=379),
 Row(Year=2017, Coal=174568, Oil=6976),
 Row(Year=2017, Coal=4508568, Oil=10278)]

In [145]:
electricty_total = electricity_earth.withColumn('totalGen',electricity_earth['Coal'] + electricity_earth['Oil'])

In [146]:
electricty_total.head(5)

[Row(Year=2018, Coal=1254542, Oil=38825, totalGen=1293367),
 Row(Year=2018, Coal=17570, Oil=1682, totalGen=19252),
 Row(Year=2018, Coal=2035, Oil=379, totalGen=2414),
 Row(Year=2017, Coal=174568, Oil=6976, totalGen=181544),
 Row(Year=2017, Coal=4508568, Oil=10278, totalGen=4518846)]

In [149]:
from pyspark.ml.feature import VectorAssembler
featureAssembler = VectorAssembler(inputCols=['Year','Coal','Oil'], outputCol='independent')

In [150]:
outputFile = featureAssembler.transform(electricty_total)

In [152]:
outputFile.show()

+----+-------+-----+--------+--------------------+
|Year|   Coal|  Oil|totalGen|         independent|
+----+-------+-----+--------+--------------------+
|2018|1254542|38825| 1293367|[2018.0,1254542.0...|
|2018|  17570| 1682|   19252|[2018.0,17570.0,1...|
|2018|   2035|  379|    2414|[2018.0,2035.0,37...|
|2017| 174568| 6976|  181544|[2017.0,174568.0,...|
|2017|4508568|10278| 4518846|[2017.0,4508568.0...|
|2017|  25337|15917|   41254|[2017.0,25337.0,1...|
+----+-------+-----+--------+--------------------+



In [154]:
finalData = outputFile.select('independent','totalGen')

In [155]:
finalData.show()

+--------------------+--------+
|         independent|totalGen|
+--------------------+--------+
|[2018.0,1254542.0...| 1293367|
|[2018.0,17570.0,1...|   19252|
|[2018.0,2035.0,37...|    2414|
|[2017.0,174568.0,...|  181544|
|[2017.0,4508568.0...| 4518846|
|[2017.0,25337.0,1...|   41254|
+--------------------+--------+



In [156]:
from pyspark.ml.regression import LinearRegression

In [157]:
train, test = finalData.randomSplit([0.7,0.3])

In [158]:
regressor = LinearRegression(featuresCol='independent', labelCol='totalGen')

In [159]:
regressor = regressor.fit(train)

22/10/31 21:12:40 WARN Instrumentation: [7368f4b5] regParam is zero, which might cause numerical instability and overfitting.


22/10/31 21:12:41 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/10/31 21:12:41 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


22/10/31 21:12:41 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [160]:
regressor.coefficients

DenseVector([-570.6127, 0.9999, 1.0045])

In [162]:
regresResult = regressor.evaluate(test)

In [163]:
regresResult.predictions.show()

+--------------------+--------+------------------+
|         independent|totalGen|        prediction|
+--------------------+--------+------------------+
|[2017.0,25337.0,1...|   41254| 41887.89314938639|
|[2017.0,174568.0,...|  181544|182117.23288962513|
|[2018.0,2035.0,37...|    2414| 2410.219358005561|
+--------------------+--------+------------------+



In [164]:
from pyspark.ml.feature import StringIndexer

In [166]:
indexer = StringIndexer(inputCol='Year',outputCol='yearIndex')
newDf = indexer.fit(electricity_earth).transform(electricity_earth)

In [168]:
newDf.show()

+----+-------+-----+---------+
|Year|   Coal|  Oil|yearIndex|
+----+-------+-----+---------+
|2018|1254542|38825|      1.0|
|2018|  17570| 1682|      1.0|
|2018|   2035|  379|      1.0|
|2017| 174568| 6976|      0.0|
|2017|4508568|10278|      0.0|
|2017|  25337|15917|      0.0|
+----+-------+-----+---------+

